In [5]:
import os  # Import the os module to get the process ID
import requests
import random
import time
import psycopg2

def quantum_random_number(min_val, max_val, count=1):
    """
    Generates random numbers using the QRNG API.
    Falls back to classical randomness if the QRNG API fails.
    
    Parameters:
        min_val (int): Minimum value (inclusive).
        max_val (int): Maximum value (inclusive).
        count (int): Number of random numbers to generate.
        
    Returns:
        tuple: A tuple containing the list of random numbers and the source ('quantum' or 'classical').
    """
    try:
        # ANU QRNG API URL
        api_url = "https://qrng.anu.edu.au/API/jsonI.php"
        response = requests.get(api_url, params={"length": count, "type": "uint8"})
        response.raise_for_status()  # Raise an error for HTTP issues
        
        data = response.json()
        if not data["success"]:
            raise RuntimeError("QRNG API returned an unsuccessful response.")
        
        # Map the random numbers to the desired range
        random_numbers = [min_val + (num % (max_val - min_val + 1)) for num in data["data"]]
        return random_numbers, 'quantum'
    except Exception:
        # Fall back to classical randomness if QRNG API fails
        random_numbers = [random.randint(min_val, max_val) for _ in range(count)]
        return random_numbers, 'classical'

def generate_powerball_ticket():
    """
    Generates a single Powerball ticket.
    
    Returns:
        tuple: A tuple containing the list of 5 white balls, a Powerball, and the source.
    """
    # Generate 5 unique white balls
    white_balls = []
    source = None
    while len(white_balls) < 5:
        number, source = quantum_random_number(1, 69, count=1)
        if number[0] not in white_balls:
            white_balls.append(number[0])
    white_balls.sort()  # Sort the white balls for display

    # Generate the Powerball
    powerball, source = quantum_random_number(1, 26, count=1)
    
    return (white_balls, powerball[0], source)

def insert_ticket_into_db(white_balls, powerball, source, process_id):
    """
    Inserts a Powerball ticket into the PostgreSQL database.
    """
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(
            dbname="rag_db",      # Replace with your database name
            user="user",          # Replace with your database username
            password="password",  # Replace with your database password
            host="192.168.0.89",  # Replace if not localhost
            port=5432             # Replace with your PostgreSQL port
        )
        cursor = conn.cursor()

        # Insert query
        insert_query = """
        INSERT INTO powerball_sessions (white_ball_1, white_ball_2, white_ball_3, white_ball_4, white_ball_5, powerball, source, process_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
        """
        # Execute the query
        cursor.execute(insert_query, (*white_balls, powerball, source, process_id))
        conn.commit()
        print("Ticket successfully inserted into the database.")
    except Exception as e:
        print(f"Database error: {e}")
    finally:
        cursor.close()
        conn.close()

def continuously_generate_powerball():
    """
    Continuously generates Powerball tickets, stores them in a database, retries on internet connection loss,
    and declares the last generated ticket as the winning ticket when interrupted.
    """
    process_id = os.getpid()  # Get the current process ID
    winning_ticket = None
    while True:
        try:
            # Generate a Powerball ticket
            winning_ticket = generate_powerball_ticket()
            white_balls, powerball, source = winning_ticket

            # Print the ticket
            print(f"Generated Ticket: White Balls: {white_balls}, Powerball: {powerball} (Source: {source}, Process ID: {process_id})")

            # Insert the ticket into the database
            insert_ticket_into_db(white_balls, powerball, source, process_id)

            time.sleep(1)  # Slight delay to avoid excessive API requests
        except KeyboardInterrupt:
            print("\nProcess interrupted by the user.")
            if winning_ticket:
                white_balls, powerball, source = winning_ticket
                print(f"The WINNING TICKET is: White Balls: {white_balls}, Powerball: {powerball} (Source: {source}, Process ID: {process_id})")
            else:
                print("No ticket was generated before the interruption.")
            break
        except Exception as e:
            print(f"\nInternet connection lost! Retrying... Error: {e}")
            time.sleep(5)  # Wait before retrying

# Start the Powerball generation process
if __name__ == "__main__":
    continuously_generate_powerball()



Generated Ticket: White Balls: [1, 4, 51, 54, 65], Powerball: 10 (Source: classical, Process ID: 6261)
Ticket successfully inserted into the database.
Generated Ticket: White Balls: [3, 10, 24, 37, 39], Powerball: 8 (Source: classical, Process ID: 6261)
Ticket successfully inserted into the database.
Generated Ticket: White Balls: [11, 13, 15, 50, 66], Powerball: 18 (Source: classical, Process ID: 6261)
Ticket successfully inserted into the database.
Generated Ticket: White Balls: [16, 26, 28, 51, 65], Powerball: 22 (Source: classical, Process ID: 6261)
Ticket successfully inserted into the database.
Generated Ticket: White Balls: [8, 9, 15, 42, 48], Powerball: 19 (Source: classical, Process ID: 6261)
Ticket successfully inserted into the database.
Generated Ticket: White Balls: [4, 18, 42, 46, 61], Powerball: 9 (Source: classical, Process ID: 6261)
Ticket successfully inserted into the database.
Generated Ticket: White Balls: [4, 6, 22, 47, 67], Powerball: 18 (Source: classical, Proce